In [1]:
import logging
from math import *
import sys
import time
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt

# Logging: Feil, Varsler, Info & Debug
logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

class IK:

    # Servos telles fra bunn til topp.
    # Lenkeparametre:
    l1 = 11.5       # Distansen fra senter av basen (servo1) til servo2: 6.10 cm.
                    # Må kanskje padde l1 litt mtp hjul og chassis
    l2 = 6.30       # Distansen fra servo2 til servo3: 10.16 cm.
    l3 = 6.10       # Distansen fra servo3 til servo4: 9.64 cm.
    l4 = 10.00      # Distansen fra servo4 aksen til tuppen på kloa: 10.0 cm.

    def __init__(self, arm_type):
        # Kan legge til prismatiske ledd her senere 
        self.arm_type = arm_type
        self.arm_type == 'arm'

    def setLinkLength(self, L1=l1, L2=l2, L3=l3, L4=l4):
        # Kan endre lengden på lenker her dersom prismatiske ledd introduseres
        # eller hvis manuell justering av parametre blir nødvendig
        self.l1 = L1
        self.l2 = L2
        self.l3 = L3
        self.l4 = L4

    def getLinkLength(self):
        # Henter ut lengdene på lenkene
        return {"L1":self.l1, "L2":self.l2, "L3":self.l3, "L4":self.l4}

    def convert_angle(angle):
    
        if angle > 90:
            return angle - 180
        return angle

    def getRotationAngle(self, coordinate_data, Alpha):
            # Gitt koordinater og pitch angle 'Alpha', returnerer vinkler for hvert ledd.
            # Returnerer 'False' hvis ingen gyldig løsning.
            # Alpha er vinkelen mellom ende-effektor og XY-planet i grader.
            # Alpha er også vinkelen mellom CD og PC.
            # P(X,Y,Z) er ende-effektoren. O er origo, projisert på bakken under servo1.
            # Punkt A er leddet mellom l1 og l2, punkt B er leddet mellom l2 og l3
            # og punkt C er leddet mellom l3 og l4. CD er ortogonal med PD og Z-aksen.
            # Vinkler er representert på følgende måte: Vinkel mellom AB og BC er ABC.

            X, Y, Z = coordinate_data

            # Theta_6 = rotasjonsvinkel til baseledd (servo1); rotasjon rundt Z_0.
            theta6 = degrees(atan2(Y, X))

            P_O = sqrt(X*X + Y*Y)                   # Distanse fra Origo til ende-effektor
            CD = self.l4 * cos(radians(Alpha))      # Lengde (X) fra ledd-C til ende-effektor (hjelpe-punkt D)
            PD = self.l4 * sin(radians(Alpha))      # Høyde (Z) fra ledd-C til ende-effektor, (hjelpe-punkt D). Alpha pos -> PD = pos, Alpha neg -> PD = neg.
            AF = P_O - CD                           # Distanse fra Origo (eller A) til hjelpe-punkt F, langs X i XZ.  
            CF = Z - self.l1 - PD                   # Høyde (Z) mellom C og hjelpe-punkt F
            AC = sqrt(pow(AF, 2) + pow(CF, 2))      # Distance mellom punkt A og C, via hjelpe-punkt F

            # Sjekker at høydeverdi er konsistent med verdensrammen.
            if round(CF, 4) < -self.l1:
                logger.debug('Høyde under 0, CF(%s)<l1(%s)', CF, -self.l1)
                return False
            if self.l2 + self.l3 < round(AC, 4): # Summen av de to sidene er mindre enn den tredje
                logger.debug('Ugyldig koblingsmekanisme, l2(%s) + l3(%s) < AC(%s)', self.l2, self.l3, AC)
                return False
            
            # Theta_4. phi_B = vinkel ABC, mellom AB og BC
            cos_ABC = round((pow(self.l2, 2) + pow(self.l3, 2) - pow(AC, 2))/(2*self.l2*self.l3), 4) # Law of cosines
            if abs(cos_ABC) > 1:
                logger.debug('Ugyldig koblingsmekanisme, abs(cos_phi_B(%s)) > 1', cos_ABC)
                return False                    # Sjekker for gyldig verdi
            ABC = acos(cos_ABC)             # Finner vinkel phi_B i [rad]
            theta4 = 180.0 - degrees(ABC)     # Konverterer til grader

            # Theta_5. 
            CAF = acos(AF / AC)         # CAF = vinkel mellom AF og CF. 
            cos_BAC = round((pow(AC, 2) + pow(self.l2, 2) - pow(self.l3, 2))/(2*self.l2*AC), 4) # Law of cosines
            if abs(cos_BAC) > 1:
                logger.debug('Ugyldig koblingsmekanisme, abs(cos_BAC(%s)) > 1', cos_BAC)
                return False            # Sjekker for gyldig verdi
            if CF < 0:                  # Hvis CF er negativ, betyr det at punkt F er over punkt C. zf flag settes til negativ for å justere vinkel CAF
                zf_flag = -1            # Hvis F er under punkt C, er CAF positiv
            else:
                zf_flag = 1
            theta5 = degrees(CAF * zf_flag + acos(cos_BAC))

            # Theta_3
            theta3 = Alpha - theta5 + theta4
            print("X: ", X, "Y: ", Y, "Z: ", Z)
            print("theta3", theta3, "theta4", theta4, "theta5", theta5, "theta6", theta6)
            #return {"theta3":theta3, "theta4":theta4, "theta5":theta5, "theta6":theta6} # Returns the angles if there is a solution
            return theta3, theta4, theta5, theta6
    
class ArmIK:
    # Pulsbredde intervall [us], vinkelintervall [deg]
    servo3Range = (500, 2500.0, 0, 180.0)       # Servo ID3: ledd C
    servo4Range = (500, 2500.0, 0, 180.0)       # Servo ID4: ledd B
    servo5Range = (500, 2500.0, 0, 180.0)       # Servo ID5: ledd A
    servo6Range = (500, 2500.0, 0, 180.0)       # Servo ID6: Base-ledd

    def __init__(self):
        self.setServoRange()

    def setServoRange(self, servo3_Range=servo3Range, servo4_Range=servo4Range, servo5_Range=servo5Range, servo6_Range=servo6Range):
        # Angir intervall for servos
        self.servo3Range = servo3_Range
        self.servo4Range = servo4_Range
        self.servo5Range = servo5_Range
        self.servo6Range = servo6_Range

        # Regner ut pulsbredde per grad i [us/deg]
        self.servo3Param = ((self.servo3Range[1] - self.servo3Range[0]) / (self.servo3Range[3] - self.servo3Range[2]))
        self.servo4Param = ((self.servo4Range[1] - self.servo4Range[0]) / (self.servo4Range[3] - self.servo4Range[2]))
        self.servo5Param = ((self.servo5Range[1] - self.servo5Range[0]) / (self.servo5Range[3] - self.servo5Range[2]))
        self.servo6Param = ((self.servo6Range[1] - self.servo6Range[0]) / (self.servo6Range[3] - self.servo6Range[2]))

    def transformAngelAdaptArm(self, theta3, theta4, theta5, theta6):
        # Konverterer vinkler fra IK_geo til korresponderende pulslengder
        # Inkludert mapping fra [0,180] til [-90,90]

        # Vinkel [deg] * 11.11 [us/deg] + (2500 + 500)/2 [us]
        servo3 = int(round(theta3 * self.servo3Param + (self.servo3Range[1] + self.servo3Range[0])/2))
        if servo3 > self.servo3Range[1] or servo3 < self.servo3Range[0]:
            # Sjekker om vinkelutslag er innenfor gyldig område
            logger.info('servo3(%s)Utenfor gyldig område(%s, %s)', servo3, self.servo3Range[0], self.servo3Range[1])
            return False
        
        servo4 = int(round(theta4 * self.servo4Param + (self.servo4Range[1] + self.servo4Range[0])/2))
        if servo4 > self.servo4Range[1] or servo4 < self.servo4Range[0]:
            # Sjekker om vinkelutslag er innenfor gyldig område
            logger.info('servo4(%s)Utenfor gyldig område(%s, %s)', servo4, self.servo4Range[0], self.servo4Range[1])
            return False
        
        servo5 = int(round((self.servo5Range[1] + self.servo5Range[0])/2 + (90.0 - theta5) * self.servo5Param)) 
        if servo5 > ((self.servo5Range[1] + self.servo5Range[0])/2 + 90*self.servo5Param) or servo5 < ((self.servo5Range[1] + self.servo5Range[0])/2 - 90*self.servo5Param):
            # Sjekker om vinkelutslag er innenfor gyldig område
            # -> Vinkel mellom [-pi/2,pi/2] ? 
            logger.info('servo5(%s)Utenfor gyldig område(%s, %s)', servo5, self.servo5Range[0], self.servo5Range[1])
            return False
        
        # Konverterer til +deg hvis deg < -90. Med eller mot klokka
        if theta6 < -(self.servo6Range[3] - self.servo6Range[2])/2:
            # Hvis theta6 < -90 [deg] -> servo6 = (180-0)/2 + 90 + (180 + theta6)
            servo6 = int(round(((self.servo6Range[3] - self.servo6Range[2])/2 + (90 + (180 + theta6))) * self.servo6Param))
        else:
            # Hvis ikke: servo6 = ((180-0)/2 -(90-theta6))* 11.11us + 500us 
            servo6 = int(round(((self.servo6Range[3] - self.servo6Range[2])/2 - (90 - theta6)) * self.servo6Param)) + self.servo6Range[0]
        if servo6 > self.servo6Range[1] or servo6 < self.servo6Range[0]:
            # Sjekker om vinkelutslag er innenfor gyldig område
            logger.info('servo6(%s)Utenfor gyldig område(%s, %s)', servo6, self.servo6Range[0], self.servo6Range[1])
            return False
        return {"servo3": servo3, "servo4": servo4, "servo5": servo5, "servo6": servo6}

    def pwm_to_angle(self, pwm_values):
    
        # First check if we got valid PWM values
        if pwm_values is False or not isinstance(pwm_values, dict):
            return False
            
        try:
            # Verify all required keys exist and are numeric
            required_keys = ['servo3', 'servo4', 'servo5', 'servo6']
            if not all(key in pwm_values for key in required_keys):
                return False
                
            # Convert to float and validate ranges
            pwms = {k: float(pwm_values[k]) for k in required_keys}
            if not (self.servo3Range[0] <= pwms['servo3'] <= self.servo3Range[1] and
                    self.servo4Range[0] <= pwms['servo4'] <= self.servo4Range[1] and
                    self.servo5Range[0] <= pwms['servo5'] <= self.servo5Range[1] and
                    self.servo6Range[0] <= pwms['servo6'] <= self.servo6Range[1]):
                return False

            # ... rest of your conversion logic ...
            
        except (ValueError, TypeError):
            return False

        # Conversion formulas (reverse of transformAngelAdaptArm)
        def clamp_angle(angle):
            """Clamp angle to ±90° range"""
            return max(-90.0, min(90.0, angle))

        # Servo 3 (direct mapping)
        theta3 = clamp_angle((pwm_values['servo3'] - (self.servo3Range[1] + self.servo3Range[0])/2) / self.servo3Param)
        
        # Servo 4 (direct mapping)
        theta4 = clamp_angle((pwm_values['servo4'] - (self.servo4Range[1] + self.servo4Range[0])/2) / self.servo4Param)
        
        # Servo 5 (special inverted mapping)
        theta5 = clamp_angle(90.0 - (pwm_values['servo5'] - (self.servo5Range[1] + self.servo5Range[0])/2) / self.servo5Param)
        
        # Servo 6 (special circular mapping)
        mid_point = (self.servo6Range[1] + self.servo6Range[0])/2
        if pwm_values['servo6'] > mid_point + 90 * self.servo6Param:
            theta6 = clamp_angle(180 - ((pwm_values['servo6']/self.servo6Param) - mid_point/self.servo6Param - 90))
        else:
            theta6 = clamp_angle(90 - (mid_point/self.servo6Param - pwm_values['servo6']/self.servo6Param))
        
        return {
            'theta3': round(theta3, 2),
            'theta4': round(theta4, 2),
            'theta5': round(theta5, 2),
            'theta6': round(theta6, 2)
        }

In [2]:
ik = IK('arm')
ak = ArmIK()
#ik.getRotationAngle((5,6,18), 0)

#ak.transformAngelAdaptArm()

# Quick call to chain them:
generator = ik
processor = ak

pwm_result = processor.transformAngelAdaptArm(*generator.getRotationAngle((10,13,10), -45))

if pwm_result is False:
    print("Error: Invalid angles received from generator")
else:
    # Now safely convert back to angles
    angles = processor.pwm_to_angle(pwm_result)
    if angles:
        print(f"Converted angles: {angles}")
    else:
        print("Error: Invalid PWM values received")


X:  10 Y:  13 Z:  10
theta3 -46.53172230563747 theta4 57.595032197775524 theta5 59.126754503413004 theta6 52.43140797117251
Converted angles: {'theta3': -46.53, 'theta4': 57.6, 'theta5': 59.13, 'theta6': 52.47}


### Funke ikke helt som den ska enda

In [26]:
from ikpy.chain import Chain
from ikpy.link import URDFLink
import numpy as np

# Define your 4-DOF chain (updated syntax)
chain = Chain(name="4DOF_Arm", links=[
    URDFLink(
        name="base",
        bounds=[-np.pi/2, np.pi/2],
        origin_translation=[0, 0, 0.115],  # Link 1 length (Z-axis)
        origin_orientation=[0, 0, 0],
        rotation=[0, 0, 1]  # Revolute Z-axis
    ),
    URDFLink(
        name="shoulder",
        bounds=[-np.pi/2, np.pi/2],
        origin_translation=[0, 0, 0],
        origin_orientation=[0, 0, 0],
        rotation=[0, 1, 0]  # Revolute Y-axis
    ),
    URDFLink(
        name="elbow",
        bounds=[-np.pi/2, np.pi/2],
        origin_translation=[0.061, 0, 0],  # Link 2 length (X-axis)
        origin_orientation=[0, 0, 0],
        rotation=[0, 1, 0]
    ),
    URDFLink(
        name="wrist",
        bounds=[-np.pi/2, np.pi/2],
        origin_translation=[0.063, 0, 0],  # Link 3 length (X-axis)
        origin_orientation=[0, 0, 0],
        rotation=[0, 1, 0]
    )
])

# Solve IK for target [x, y, z]
target_position = [0.01, 0.5, 0.02]  # Example target
joint_angles = chain.inverse_kinematics(
    target_position=target_position,
    initial_position=[0]*4,  # Neutral start
    max_iter=100
)

# Convert to degrees and print (excluding first element if fixed base)
print(f"Joint angles (deg): {np.rad2deg(joint_angles[:])}")

Joint angles (deg): [8.88312759e+01 1.07592367e+01 3.85771868e-03 0.00000000e+00]
